<a href="https://colab.research.google.com/github/ahsan3274/Fellowship.ai/blob/main/ai_fellowship.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opendatasets
!pip install pandas
import opendatasets as od
od.download(
    "https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews"
)

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: nasquamesse
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews


100%|██████████| 25.7M/25.7M [00:00<00:00, 115MB/s] 


In [ ]:
import pandas as pd
df = pd.read_csv('/content/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv')
print(df.head())
print(df.info())
print(df['sentiment'].value_counts())

                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB
None
sentiment
positive    25000
negative    25000
Name: count, dtype: int64


In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Download necessary NLTK data files
nltk.download('stopwords')
nltk.download('wordnet')

# Initialize lemmatizer and stop words
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    # Lowercase the text
    text = text.lower()
    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    # Remove punctuation and special characters
    text = re.sub(r'[^\w\s]', '', text)
    # Remove stop words and lemmatize the text
    words = [lemmatizer.lemmatize(word) for word in text.split() if word not in stop_words]
    return ' '.join(words)

# Apply preprocessing to the reviews
df['review'] = df['review'].apply(preprocess_text)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize TF-IDF vectorizer
tfidf = TfidfVectorizer(max_features=5000)

# Fit and transform the reviews
X = tfidf.fit_transform(df['review']).toarray()
y = df['sentiment'].apply(lambda x: 1 if x == 'positive' else 0).values


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train logistic regression model
model = LogisticRegression()
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)


In [ ]:
# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.4f}')

# Display classification report
print('Classification Report:')
print(classification_report(y_test, y_pred))

# Display confusion matrix
print('Confusion Matrix:')
print(confusion_matrix(y_test, y_pred))


Accuracy: 0.8866
Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.87      0.88      4961
           1       0.88      0.90      0.89      5039

    accuracy                           0.89     10000
   macro avg       0.89      0.89      0.89     10000
weighted avg       0.89      0.89      0.89     10000

Confusion Matrix:
[[4327  634]
 [ 500 4539]]


LSTM networks are specifically designed to handle sequential data and maintain long-term dependencies. In the context of sentiment analysis, understanding the sequence of words and their context within a review is crucial for accurately determining sentiment. Unlike traditional neural networks, LSTMs can capture the temporal dependencies between words in a sentence. Standard Recurrent Neural Networks (RNNs) struggle with the vanishing gradient problem, making it difficult to learn long-term dependencies. LSTMs overcome this issue through their unique architecture, which includes gates (input, forget, and output gates) that regulate the flow of information. This allows LSTMs to remember relevant information over long sequences and forget irrelevant details, leading to better performance on tasks requiring context retention. Sentiment analysis relies heavily on context to interpret the sentiment of words correctly. For example, the meaning of "not bad" is different from "bad." LSTM networks excel at capturing such contextual nuances by maintaining an internal state that evolves as they process each word in the sequence. This ability to understand the context makes LSTMs particularly well-suited for sentiment analysis.

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Tokenize and pad the sequences
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(df['review'])
X = tokenizer.texts_to_sequences(df['review'])
X = pad_sequences(X, maxlen=500)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build the LSTM model with Dropout layers
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=500))
model.add(LSTM(units=128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Use early stopping to avoid overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

# Train the model with regularization and early stopping
model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test), callbacks=[early_stopping])

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Accuracy: {accuracy:.4f}')


Epoch 1/10
625/625 [==============================] - 1126s 2s/step - loss: 0.3596 - accuracy: 0.8408 - val_loss: 0.2764 - val_accuracy: 0.8875
Epoch 2/10
625/625 [==============================] - 1096s 2s/step - loss: 0.2511 - accuracy: 0.9006 - val_loss: 0.3074 - val_accuracy: 0.8654
Epoch 3/10
313/313 [==============================] - 54s 172ms/step - loss: 0.2764 - accuracy: 0.8875
Accuracy: 0.8875


The results show that the model performs well on the training data, with continuous improvements in training accuracy and decreases in training loss. The early stopping mechanism worked as intended, stopping training when the validation loss did not improve for two consecutive epochs, thereby preventing further overfitting. The validation accuracy peaked early and remained high, indicating that the model maintained its generalization ability throughout the training. The slight fluctuations in validation performance suggest that while the model is robust, there might be room for fine-tuning or additional regularization techniques to improve stability and prevent any overfitting. Overall, achieving an accuracy of 88.75% on the test set is a strong indicator that the LSTM model is effective for sentiment analysis on the IMDB dataset.